In [2]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("Retail.csv")

In [4]:
df.head()

,TransactionID,CustomerID,Gender,Age,City,ProductCategory,Quantity,Price,PurchaseDate,PaymentMode,TotalAmount
0,T00001,C2824,Female,33,Bengaluru,Fashion,3,4479,2023-03-15,Cash,13437
1,T00002,C1409,Other,20,Pune,Books,5,1051,2023-01-22,Card,5255
2,T00003,C5506,Other,47,Pune,Furniture,1,111,2023-04-18,Wallet,111
3,T00004,C5012,Other,21,Hyderabad,Groceries,5,2946,2023-08-09,Cash,14730
4,T00005,C4657,Female,41,Chennai,Sports,1,3123,2023-09-23,Wallet,3123


In [5]:
#inspecting the dataset
print("Initial Shape", df.shape)
print("\nData Types: \n", df.dtypes)

Initial Shape (2000, 11)

Data Types: 
 TransactionID      object
CustomerID         object
Gender             object
Age                 int64
City               object
ProductCategory    object
Quantity            int64
Price               int64
PurchaseDate       object
PaymentMode        object
TotalAmount         int64
dtype: object


In [6]:
#missing values and duplicates
print("\nTotal Missing Values: \n", df.isnull().sum())
print("\nDuplicate Rows:",df.duplicated().sum())


Total Missing Values: 
 TransactionID      0
CustomerID         0
Gender             0
Age                0
City               0
ProductCategory    0
Quantity           0
Price              0
PurchaseDate       0
PaymentMode        0
TotalAmount        0
dtype: int64

Duplicate Rows: 0


In [7]:
# Check if there are missing values in TransactionID or ProductCategory
missing_rows = df[df[['TransactionID', 'ProductCategory']].isnull().any(axis=1)]

# Show how many rows are missing
print("Number of rows with missing TransactionID or ProductCategory:", len(missing_rows))

# Optionally display the actual rows
print(missing_rows)

Number of rows with missing TransactionID or ProductCategory: 0
Empty DataFrame
Columns: [TransactionID, CustomerID, Gender, Age, City, ProductCategory, Quantity, Price, PurchaseDate, PaymentMode, TotalAmount]
Index: []


In [8]:
#not nececessary to run as theres no missing transcation ID or product category
#handle missing values
df['Age'] = df['Age'].fillna(df['Age'].median())
if 'City' in df.columns:
    df['City']= df['City'].fillna(df['City'].mode()[0])

#drop rows with missing transcation ID or product category
df = df.dropna(subset = ['TransactionID', 'ProductCategory'])

In [9]:
#data cleaning dropping duplicates
df = df.drop_duplicates()

In [11]:
df.head(10)

,TransactionID,CustomerID,Gender,Age,City,ProductCategory,Quantity,Price,PurchaseDate,PaymentMode,TotalAmount
0,T00001,C2824,Female,33,Bengaluru,Fashion,3,4479,2023-03-15,Cash,13437
1,T00002,C1409,Other,20,Pune,Books,5,1051,2023-01-22,Card,5255
2,T00003,C5506,Other,47,Pune,Furniture,1,111,2023-04-18,Wallet,111
3,T00004,C5012,Other,21,Hyderabad,Groceries,5,2946,2023-08-09,Cash,14730
4,T00005,C4657,Female,41,Chennai,Sports,1,3123,2023-09-23,Wallet,3123
5,T00006,C3286,Male,63,Delhi,Furniture,2,2758,2023-05-23,Card,5516
6,T00007,C2679,Other,30,Pune,Electronics,5,781,2023-10-22,Card,3905
7,T00008,C9935,Female,35,Hyderabad,Books,5,4599,2023-11-12,Cash,22995
8,T00009,C2424,Female,41,Jaipur,Books,5,2263,2023-06-10,Cash,11315
9,T00010,C7912,Male,70,Lucknow,Groceries,1,1194,2023-02-20,UPI,1194


In [12]:
if 'Gender' in df.columns:
    # 1. Check for missing values
    print("Missing values:", df['Gender'].isna().sum())
    # 2. Check for leading/trailing spaces
    with_spaces = df[df['Gender'].astype(str) != df['Gender'].astype(str).str.strip()]
    print("Rows with extra spaces:\n", with_spaces['Gender'].unique())
    # 3. Check for lowercase values
    lowercase = df[df['Gender'].astype(str) == df['Gender'].astype(str).str.lower()]
    print("Rows with lowercase entries:\n", lowercase['Gender'].unique())
    # 4. Check for unusually short values (e.g., 1–2 characters like 'm' or 'f')
    short_values = df[df['Gender'].astype(str).str.len() <= 2]
    print("Short values:\n", short_values['Gender'].unique())
    # 5. Show all unique values to review overall messiness
    print("All unique Gender values:\n", df['Gender'].unique())

Missing values: 0
Rows with extra spaces:
 []
Rows with lowercase entries:
 []
Short values:
 []
All unique Gender values:
 ['Female' 'Other' 'Male']


In [13]:
#standarize gender values
if 'Gender' in df.columns:
    df['Gender'] = df['Gender'].str.strip().str.lower().map({
        'm': 'Male', 'male': 'Male', 'f': 'Female', 'female': 'Female'
    }).fillna(df['Gender'])


In [14]:
#to check how many rows have non postive values
print("Rows with Quantity <= 0:", (df['Quantity'] <= 0).sum())
print("Rows with Price <= 0:", (df['Price'] <= 0).sum())


Rows with Quantity <= 0: 0
Rows with Price <= 0: 0


In [16]:
#check if a row has problems
b_rows = df[(df['Quantity'] <= 0) | (df['Price'] <= 0)]
print(b_rows)

Empty DataFrame
Columns: [TransactionID, CustomerID, Gender, Age, City, ProductCategory, Quantity, Price, PurchaseDate, PaymentMode, TotalAmount]
Index: []


In [20]:
print("The result is an empty DataFrame as nothing matched the condition, "
      "where Quantity <= 0 or Price <= 0. The list of columns is shown but there are no rows (Index: []).")

The result is an empty DataFrame as nothing matched the condition, where Quantity <= 0 or Price <= 0. The list of columns is shown but there are no rows (Index: []).


In [21]:
# Remove negative/zero Quantity and Price
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

In [23]:
print(df['PurchaseDate'].head())

0    2023-03-15
1    2023-01-22
2    2023-04-18
3    2023-08-09
4    2023-09-23
Name: PurchaseDate, dtype: object


In [24]:
# Derive TotalAmount if missing or incorrect
df['TotalAmount'] = df['Quantity'] * df['Price']
# Convert PurchaseDate to datetime
df['PurchaseDate'] = pd.to_datetime(df['PurchaseDate'], errors='coerce')

In [25]:
# Extract Month and DayOfWeek
df['Month'] = df['PurchaseDate'].dt.month
df['DayOfWeek'] = df['PurchaseDate'].dt.day_name()

In [33]:
# Ensure Age is numeric (bad values become NaN)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')

# Create AgeGroup column using pd.cut
df['AgeGroup'] = pd.cut(
    df['Age'],
    bins=[0, 17, 25, 40, 60, 120],   # Age ranges
    labels=["<18", "18-25", "26-40", "41-60", "60+"],  # Group labels
    right=True                        # Include right edge (e.g., 25 goes into 18-25)
)

# Optional: check result
print(df[['Age', 'AgeGroup']].head(10))

   Age AgeGroup
0   33    26-40
1   20    18-25
2   47    41-60
3   21    18-25
4   41    41-60
5   63      60+
6   30    26-40
7   35    26-40
8   41    41-60
9   70      60+


In [28]:
#shows the first few values of age group
print(df['AgeGroup'].head())

0    26-40
1    18-25
2    41-60
3    18-25
4    41-60
Name: AgeGroup, dtype: object


In [29]:
#see all unique groups
print(df['AgeGroup'].unique())

['26-40' '18-25' '41-60' '60+']


In [32]:
#count how many rows per group
print(df['AgeGroup'].value_counts())

AgeGroup
41-60    791
26-40    546
60+      387
18-25    276
Name: count, dtype: int64


In [35]:
under18 = df[(df['Age'] >= 0) & (df['Age'] < 18)]
print(under18)

Empty DataFrame
Columns: [TransactionID, CustomerID, Gender, Age, City, ProductCategory, Quantity, Price, PurchaseDate, PaymentMode, TotalAmount, Month, DayOfWeek, AgeGroup]
Index: []


In [36]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Encode categorical variables
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['City'] = le.fit_transform(df['City'])
df['ProductCategory'] = le.fit_transform(df['ProductCategory'])
df['PaymentMode'] = le.fit_transform(df['PaymentMode'])
df['DayOfWeek'] = le.fit_transform(df['DayOfWeek'])

# Normalize numerical columns
scaler = MinMaxScaler()
df[['Age','Price','TotalAmount']] = scaler.fit_transform(df[['Age','Price','TotalAmount']])

In [37]:
# Check for remaining missing values
print(df.isnull().sum())


TransactionID      0
CustomerID         0
Gender             0
Age                0
City               0
ProductCategory    0
Quantity           0
Price              0
PurchaseDate       0
PaymentMode        0
TotalAmount        0
Month              0
DayOfWeek          0
AgeGroup           0
dtype: int64


In [38]:
# Save cleaned dataset
df.to_csv("Retail_Cleaned.csv", index=False)
print("Cleaned dataset Retail_Cleaned.csv")

Cleaned dataset Retail_Cleaned.csv
